In [5]:
#% pip install -q pyterrier_dr

In [6]:
import pyterrier as pt

In [7]:
bm25 = pt.Artifact.from_hf("pyterrier/vaswani.terrier").bm25()
textl = pt.text.get_text(pt.get_dataset("irds:vaswani"))

Java started (triggered by TerrierIndex.index_ref) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [8]:
from sklearn.ensemble import RandomForestRegressor
bm25 >> (bm25 ** bm25) >> pt.ltr.apply_learned_model(RandomForestRegressor())

(TerrierRetr(BM25) >> (TerrierRetr(BM25) ** TerrierRetr(BM25)) >> <pyterrier.ltr.RegressionTransformer object at 0x398b1c080>)

In [9]:
import pyterrier_dr
model = pyterrier_dr.TctColBert()
model

TctColBert('castorini/tct_colbert-msmarco')

In [10]:
flex = pyterrier_dr.FlexIndex("./dr_index")
index_pipe = model >> flex
index_pipe

(TctColBert('castorini/tct_colbert-msmarco') >> FlexIndex('dr_index'))

In [11]:
index_pipe = model >> flex.indexer()
index_pipe

(TctColBert('castorini/tct_colbert-msmarco') >> FlexIndex('dr_index').indexer(mode=<IndexingMode.create: 'create'>))

In [12]:
retr_pipe = model >> flex.np_retriever()
retr_pipe

(TctColBert('castorini/tct_colbert-msmarco') >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a92690>)

In [13]:
flex.np_retriever()

In [14]:
flex.vec_loader()

In [15]:
model >> flex.np_retriever(drop_query_vec=False) >> pyterrier_dr.prf.VectorPrf() >> flex.np_retriever(drop_query_vec=False)

(TctColBert('castorini/tct_colbert-msmarco') >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a92ae0> >> VectorPrf(alpha=1, beta=0.2, k=3) >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a91370>)

In [16]:
pyterrier_dr.prf.VectorPrf()

VectorPrf(alpha=1, beta=0.2, k=3)

In [19]:
prf_pipe = model >> flex.np_retriever(drop_query_vec=False) >> flex.vec_loader() >> pyterrier_dr.prf.VectorPrf() >> flex.np_retriever(drop_query_vec=False)
prf_pipe

(TctColBert('castorini/tct_colbert-msmarco') >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a91910> >> <pyterrier_dr.flex.np_retr.NumpyVectorLoader object at 0x398b53c50> >> VectorPrf(alpha=1, beta=0.2, k=3) >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a92f00>)

In [20]:
prf_pipe.compile()

(TctColBert('castorini/tct_colbert-msmarco') >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a91100> >> <pyterrier_dr.flex.np_retr.NumpyVectorLoader object at 0x398b53c50> >> VectorPrf(alpha=1, beta=0.2, k=3) >> <pyterrier_dr.flex.np_retr.NumpyRetriever object at 0x399a92f00>)

If you mouseover the first NumpyRetriever, it now will only retrieve 3 results, not 1000; the second one is unchanged